In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime,timedelta
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import scipy.optimize
import mysql.connector
import pytz

In [6]:
from dotenv import load_dotenv, find_dotenv


# CONEXÃO COM BANCO DE DADOS
load_dotenv(find_dotenv())
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="bolsa_valores"
)
cursor = conn.cursor()




In [10]:
x = pd.read_sql("SELECT DISTINCT SYMBOL FROM cotacao", conn)
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
symbol_list_ibrx = symbol_list_ibrx + x['SYMBOL'].values.tolist()
symbol_list_ibrx = set(symbol_list_ibrx)
symbol_list_ibrx
df_final = pd.DataFrame()
for ativo in symbol_list_ibrx:
    if(ativo in x):
        chamada_api = yf.Ticker(ativo + ".SA").history(period='1wk')
    else:    
        chamada_api = yf.Ticker(ativo + ".SA").history(start = date(2020, 3, 23))
    df_hist = pd.DataFrame(chamada_api).reset_index()
    df_hist["Symbol"] = ativo
    df_hist = df_hist[["Date","Symbol","Open","High","Low","Close","Volume","Dividends"]]
    df_hist["Date"] = df_hist["Date"].dt.tz_convert(pytz.utc).dt.date
    df_hist["Date"] = df_hist["Date"].astype(str)
    df_final = pd.concat([df_hist, df_final],axis=0)

tuplas = tuple(df_final.itertuples(index=False, name=None))
query = "INSERT IGNORE INTO cotacao  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query, tuplas)
conn.commit()
conn.close()

C:\Users\igorb\AppData\Local\Temp\ipykernel_416\877495157.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  x = pd.read_sql("SELECT DISTINCT SYMBOL FROM cotacao", conn)


Got error from yahoo api for ticker ENGI1.SA, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ENGI1.SA: No timezone found, symbol may be delisted


KeyError: "['Dividends'] not in index"